In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
!wget --keep-session-cookies --save-cookies=cookies.txt --post-data 'username=avilaJorge&password=C323781C&submit=Login' https://www.cityscapes-dataset.com/login/
!wget --load-cookies cookies.txt --content-disposition https://www.cityscapes-dataset.com/file-handling/?packageID=1

In [0]:
!wget --keep-session-cookies --save-cookies=cookies.txt --post-data 'username=avilaJorge&password=C323781C&submit=Login' https://www.cityscapes-dataset.com/login/
!wget --load-cookies cookies.txt --content-disposition https://www.cityscapes-dataset.com/file-handling/?packageID=3

In [0]:
import zipfile
dt_path = './datasets/cityscapes/'

In [0]:
with zipfile.ZipFile('gtFine_trainvaltest.zip', 'r') as zip_ref:
    zip_ref.extractall(dt_path)

In [0]:
with zipfile.ZipFile('leftImg8bit_trainvaltest.zip', 'r') as zip_ref:
    zip_ref.extractall(dt_path)

In [0]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime → "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')


In [0]:
%matplotlib inline

import numpy as np

from torchvision import utils
from dataloader import *
from utils_2 import *
import torchvision
import torch.nn.functional as F
import torch.optim as optim
import torch.nn as nn
from torch.autograd import Variable
import torch
import time
from datetime import datetime
from PIL import Image
import matplotlib.pyplot as plt
import time
import os
import copy

plt.ion()   # interactive mode

In [0]:
class FCN8S(nn.Module):

    def __init__(self, n_class):
        super(FCN8S, self).__init__()
        self.n_class = n_class
        self.relu    = nn.ReLU(inplace=False)
        self.pool = nn.MaxPool2d(2, stride=2) 
        self.conv11 = nn.Conv2d(3,  64, kernel_size=3, stride=1, padding=100, dilation=1)
        self.conv12 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1, dilation=1)

        self.conv21 = nn.Conv2d(64,  128, kernel_size=3, stride=1, padding=1, dilation=1)
        self.conv22 = nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1, dilation=1)

        self.conv31 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1, dilation=1)
        self.conv32 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1, dilation=1)
        self.conv33 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1, dilation=1)

        self.conv41 = nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1, dilation=1)
        self.conv42 = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1, dilation=1)
        self.conv43 = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1, dilation=1)

        self.conv51 = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1, dilation=1)
        self.conv52 = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1, dilation=1)
        self.conv53 = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1, dilation=1)

        self.conv6 = nn.Conv2d(512, 4096, kernel_size=7, padding=0)
        self.dropout6 = nn.Dropout2d(p=0.5, inplace=True)

        self.conv7 = nn.Conv2d(4096, 4096, kernel_size=1, padding=0)
        self.dropout7 = nn.Dropout(p=0.5, inplace=True)

        self.clsfr1 = nn.Conv2d(4096, self.n_class, kernel_size=1, padding=0)
        self.clsfr2 = nn.Conv2d(512, self.n_class, kernel_size=1, padding=0)
        self.clsfr3 = nn.Conv2d(256, self.n_class, kernel_size=1, padding=0)

        self.deconv1 = nn.ConvTranspose2d(self.n_class, self.n_class, kernel_size=4, stride=2, padding=0)
        self.deconv2 = nn.ConvTranspose2d(self.n_class, self.n_class, kernel_size=4, stride=2, padding=0)
        self.deconv3 = nn.ConvTranspose2d(self.n_class, self.n_class, kernel_size=16, stride=8, padding=0)

    def __conv_relu(self, input, conv, pool=None):
        if pool is None:
          return self.relu(conv(input))
        else:
          return pool(self.relu(conv(input)))


    def forward(self, x):
        z = self.__conv_relu(x, self.conv11)
        z = self.__conv_relu(z, self.conv12, self.pool)

        z = self.__conv_relu(z, self.conv21)
        z = self.__conv_relu(z, self.conv22, self.pool)

        z = self.__conv_relu(z, self.conv31)
        z = self.__conv_relu(z, self.conv32)
        sp3 = self.__conv_relu(z, self.conv33, self.pool)

        z = self.__conv_relu(sp3, self.conv41)
        z = self.__conv_relu(z, self.conv42)
        sp4 = self.__conv_relu(z, self.conv43, self.pool)

        z = self.__conv_relu(sp4, self.conv51)
        z = self.__conv_relu(z, self.conv52)
        z = self.__conv_relu(z, self.conv53, self.pool)

        z = self.dropout6(self.__conv_relu(z, self.conv6))

        sfr = self.dropout7(self.__conv_relu(z, self.conv7))

        ups2 = self.deconv1(self.clsfr1(sfr))
        sp4  = self.clsfr2(sp4)
        sp3  = self.clsfr3(sp3)
        cropped = sp4[:, :, 10:(10 + ups2.size()[2]), 10:(10 + ups2.size()[3])]
        ups4 = self.deconv2(sp4[:, :, 10:(10 + ups2.size()[2]), 10:(10 + ups2.size()[3])] + ups2)
        ups3 = self.deconv3(sp3[:, :, 18:(18 + ups4.size()[2]), 18:(18 + ups4.size()[3])] + ups4)
        score = ups3[:, :, 56: (56 + x.size(2)), 56: (56 + x.size(3))].contiguous()
        return score

In [0]:
class FCN_bak(torch.nn.Module):

    def __init__(self, n_class):
        super(FCN_bak, self).__init__()
        self.n_class = n_class
        self.conv1   = nn.Conv2d(3, 32, kernel_size=(3,5), stride=(2,4), padding=1, dilation=1)
        self.bnd1    = nn.BatchNorm2d(32)
        self.conv2   = nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1, dilation=1)
        self.bnd2    = nn.BatchNorm2d(64)
        self.conv3   = nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1, dilation=1)
        self.bnd3    = nn.BatchNorm2d(128)
        self.conv4   = nn.Conv2d(128,256, kernel_size=3, stride=2, padding=1, dilation=1)
        self.bnd4    = nn.BatchNorm2d(256)
        self.conv5   = nn.Conv2d(256, 512, kernel_size=3, stride=2, padding=1, dilation=1)
        self.bnd5    = nn.BatchNorm2d(512)
        self.relu    = nn.ReLU(inplace=True)
        
        self.deconv1 = nn.ConvTranspose2d(512, 256, kernel_size=3, stride=2, padding=1, dilation=1, output_padding=1)
        self.bn1     = nn.BatchNorm2d(256)
        self.deconv2 = nn.ConvTranspose2d(256, 128, kernel_size=3, stride=2, padding=1, dilation=1, output_padding=1)
        self.bn2     = nn.BatchNorm2d(128)
        self.deconv3 = nn.ConvTranspose2d(128, 64, kernel_size=3, stride=2, padding=1, dilation=1, output_padding=1)
        self.bn3     = nn.BatchNorm2d(64)
        self.deconv4 = nn.ConvTranspose2d(64, 32, kernel_size=3, stride=2, padding=1, dilation=1, output_padding=1)
        self.bn4     = nn.BatchNorm2d(32)
        self.deconv5  = nn.ConvTranspose2d(32, 3, kernel_size=(3, 5), stride=(2,4), padding=1, dilation=1, output_padding=1)
        self.bn5= nn.BatchNorm2d(3)
        self.classifier = nn.Conv2d(3,n_class, kernel_size=1, stride=1, padding=0, dilation=1)
        
    def forward(self, x):
        pool = nn.MaxPool2d(2, stride=2,return_indices = True)
        unpool = nn.MaxUnpool2d(2, stride=2)
        
        x1, indice1 = pool(self.relu(self.conv1(x)))
        x2, indice2 = pool(self.relu(self.conv2(self.bnd1(x1))))
        x3, indice3 = pool(self.relu(self.conv3(self.bnd2(x2))))
        x4, indice4 = pool(self.relu(self.conv4(self.bnd3(x3))))
        x5, indice5 = pool(self.relu(self.conv5(self.bnd4(x4))))
        
        z1 = self.deconv1(self.bnd5(self.relu(unpool((x5), indice5))))
        z2 = self.deconv2(self.bn1(self.relu(unpool((z1), indice4))))
        z3 = self.deconv3(self.bn2(self.relu(unpool((z2), indice3))))
        z4 = self.deconv4(self.bn3(self.relu(unpool((z3), indice2))))
        z5 = self.deconv5(self.bn4(self.relu(unpool((z4), indice1))))
        
        out_decoder = self.classifier(self.bn5(z5))                  

        return out_decoder  # size=(N, n_class, x.H/1, x.W/1)

In [0]:
class FCN_TL(torch.nn.Module):

    def __init__(self, n_class):
        super(FCN_TL, self).__init__()
        self.n_class = n_class
        self.relu    = nn.ReLU(inplace=True)
        # Parameters of newly constructed modules have requires_grad=True by default
        self.model_conv = torchvision.models.resnet50(pretrained=True)
        self.in_ftrs = 1000 
        for param in self.model_conv.parameters():
          param.requires_grad = False
        
        self.conv1   = nn.Conv2d(self.in_ftrs, 512, kernel_size=2, stride=1, padding=1, dilation=1)
        self.cbnd1   = nn.BatchNorm2d(512)
        self.conv2   = nn.Conv2d(512, 256, kernel_size=3, stride=1, padding=2, dilation=1)
        self.cbnd2   = nn.BatchNorm2d(256)
        self.deconv1 = nn.ConvTranspose2d(256, 128, kernel_size=3, stride=2, padding=1, dilation=1, output_padding=1)
        self.bn1     = nn.BatchNorm2d(128)
        self.deconv2 = nn.ConvTranspose2d(128, 64, kernel_size=3, stride=2, padding=1, dilation=1, output_padding=1)
        self.bn2     = nn.BatchNorm2d(64)
        self.deconv3 = nn.ConvTranspose2d(64, 32, kernel_size=3, stride=2, padding=1, dilation=1, output_padding=1)
        self.bn3     = nn.BatchNorm2d(32)
        self.deconv4 = nn.ConvTranspose2d(32, 16, kernel_size=5, stride=4, padding=1, dilation=1, output_padding=1)
        self.bn4     = nn.BatchNorm2d(16)
        self.deconv5 = nn.ConvTranspose2d(16, 8, kernel_size=5, stride=4, padding=1, dilation=1, output_padding=1)
        self.bn5     = nn.BatchNorm2d(8)
        self.deconv6 = nn.ConvTranspose2d(8, 4, kernel_size=(3,5), stride=(2,4), padding=1, dilation=1, output_padding=1)
        self.bn6     = nn.BatchNorm2d(4)
        self.classifier = nn.Conv2d(4,self.n_class, kernel_size=1, stride=1, padding=0, dilation=1)
        
    def forward(self, x):
        z = self.model_conv(x).unsqueeze_(-1).unsqueeze_(-1)
        z = self.cbnd1(self.relu(self.conv1(z)))
        z = self.cbnd2(self.relu(self.conv2(z)))
        z = self.bn1(self.relu(self.deconv1(z)))
        z = self.bn2(self.relu(self.deconv2(z)))
        z = self.bn3(self.relu(self.deconv3(z)))
        z = self.bn4(self.relu(self.deconv4(z)))
        z = self.bn5(self.relu(self.deconv5(z)))
        z = self.bn6(self.relu(self.deconv6(z)))
        
        out_decoder = self.classifier(z)                  

        return out_decoder  # size=(N, n_class, x.H/1, x.W/1)

In [0]:
# def imshow_test(img):
#     H = img.size()[0]
#     W = img.size()[1]
#     test_colored = torch.zeros(H,W,3)
#     for row in range(H):
#       for col in range(W):
#         test_colored[row][col][0] =  labels_classes[img[row][col]].color[0]
#         test_colored[row][col][1] =  labels_classes[img[row][col]].color[1]
#         test_colored[row][col][2] =  labels_classes[img[row][col]].color[2]
#     print(img.size())   # unnormalize
#     npimg = img.numpy()
#     plt.imshow(npimg)
#     plt.show()

def imshow_test(img, original, fn):
    H = int(img.size()[0])
    W = int(img.size()[1])
    test_colored = np.zeros((H,W,3),dtype=np.uint8)

    for row in range(H):
      for col in range(W):
        test_colored[row][col][0] =  labels_classes[img[row][col]].color[0]
        test_colored[row][col][1] =  labels_classes[img[row][col]].color[1]
        test_colored[row][col][2] =  labels_classes[img[row][col]].color[2]

    test = Image.fromarray(test_colored)
    blended = Image.blend(original, test, alpha=0.5)#change alpha will change the transparency
    display(blended)
    blended.save(fn)

def test_img(data_loader, mdl):

    with torch.no_grad():
        softmax = nn.Softmax(dim=1)
        ts = time.time()
        outputs = []
        output_labels = []
        print("Starting Testing")
        
        for iter, (X, tar, Y) in enumerate(data_loader):

            inputs = X.to(device)
            labels_cat = Y.to(device)

            outputs = softmax(mdl(inputs))

            output_labels = outputs.argmax(dim=1)

        print("Finished evaluation. Time elapsed %f\n" % (time.time() - ts))
        return outputs, output_labels

In [0]:
num_wrkrs = 4
test_dataset_resized = CityScapesDatasetResize(csv_file='img_drive.csv')
test_dataset = CityScapesDataset(csv_file='img_drive.csv')
test_resized_loader = DataLoader(dataset=test_dataset_resized,
                          batch_size=1,
                          num_workers=num_wrkrs,
                          shuffle=True)
test_loader = DataLoader(dataset=test_dataset,
                          batch_size=1,
                          num_workers=num_wrkrs,
                          shuffle=True)

In [0]:
val_dataset = CityScapesValidationDataset(csv_file='val_local.csv')
val_loader = DataLoader(dataset=val_dataset,
                          batch_size=4,
                          num_workers=num_wrkrs,
                          shuffle=True)

In [0]:
def evaluate(data_loader, mdl, scale=False):
    rprt_ids = {
      'building': [],
      'traffic sign': [],
      'person': [],
      'car': [],
      'bicycle': []
    }
    with torch.no_grad():

        softmax = nn.Softmax(dim=1)
        
        for iter, (X, tar, Y) in enumerate(data_loader):

            inputs = X.to(device)
            labels_cat = Y.to(device)

            outputs = softmax(mdl(inputs))
            if scale:
              outputs = torch.nn.functional.interpolate(input=outputs, size=(1024,2048))

            output_labels = outputs.argmax(dim=1)

            ious, rprt = iou(output_labels, labels_cat)
            print(rprt)
            for cls in rprt.keys():
              rprt_ids[cls].append(rprt[cls])
    return rprt_ids
        

In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
def print_GPU_stats():
    print("total GPU Mem: ", torch.cuda.get_device_properties(device).total_memory)
    print("total GPU Cached: ", torch.cuda.memory_cached(device))
    print("total GPU Allocated: ", torch.cuda.memory_allocated(device))
    print("Available GB: ", (torch.cuda.get_device_properties(device).total_memory - torch.cuda.memory_allocated(device))/(10**9))
print_GPU_stats()

In [0]:


original_image = Image.open("berlin_000000_000019_leftImg8bit.png")
epochs     = 100
start_epoch = 1
path = '/content/drive/My Drive/CSE253/PA3/FCN8S/'
criterion = torch.nn.CrossEntropyLoss()
fcn_model = FCN8S(n_class=34)

fcn_model.load_state_dict(torch.load(path + 'best_model_02_16_07_22.pt'))
fcn_model = fcn_model.to(device)
optimizer = optim.Adam(fcn_model.parameters(), lr=5e-5, weight_decay=1e-5)

imgs, _, labels = next(iter(test_loader))
imgs_resized, _, labels_resized = next(iter(test_resized_loader))

# imshow_test(labels.squeeze(0))

# img, lbls = test_img(test_loader)
# imshow_test(lbls.squeeze(0).cpu())

# resized_img, resized_lbls = test_img(test_resized_loader, fcn_model)
# imshow_test(resized_lbls.squeeze(0).cpu())
# print(resized_lbls.size())
# ups_img = torch.nn.functional.interpolate(input=resized_lbls.unsqueeze(0).float(), size=(1024, 2048))
# print(ups_img.size())
# imshow_test(ups_img.squeeze(0).squeeze().long().cpu(), original_image, 'FCN8S.png')
rprt = evaluate(val_loader, fcn_model, scale=True)
for ky in rprt.keys():
  print(ky, np.nanmean(rprt[ky]))

In [0]:
val_dataset_pln = CityScapesDataset(csv_file='val_local.csv')
val_ldr_pln = DataLoader(dataset=val_dataset_pln,
                          batch_size=4,
                          num_workers=4,
                          shuffle=True)

In [0]:


original_image = Image.open("berlin_000000_000019_leftImg8bit.png")
epochs     = 100
start_epoch = 1
path = '/content/drive/My Drive/CSE253/PA3/Base_TL/'
criterion = torch.nn.CrossEntropyLoss()
fcn_model_TL = FCN_TL(n_class=34)

fcn_model_TL.load_state_dict(torch.load(path + 'best_model_02_15_21_27.pt'))
fcn_model_TL = fcn_model_TL.to(device)
optimizer = optim.Adam(fcn_model_TL.parameters(), lr=5e-5, weight_decay=1e-5)

imgs, _, labels = next(iter(test_loader))

img, lbls = test_img(test_loader, fcn_model_TL)
print(lbls.size())
imshow_test(lbls.squeeze(0).long().cpu(), original_image, 'FCN_TL.png')
rprt = evaluate(val_ldr_pln, fcn_model_TL)
for ky in rprt.keys():
  print(ky, np.nanmean(rprt[ky]))

In [0]:


original_image = Image.open("berlin_000000_000019_leftImg8bit.png")
epochs     = 100
start_epoch = 1
path = '/content/drive/My Drive/CSE253/PA3/Base_LR00005/'
criterion = torch.nn.CrossEntropyLoss()
fcn_model_bs = FCN_bak(n_class=34)

fcn_model_bs.load_state_dict(torch.load(path + 'best_model_02_14_11_16.pt'))
fcn_model_bs = fcn_model_bs.to(device)
optimizer = optim.Adam(fcn_model_bs.parameters(), lr=5e-5, weight_decay=1e-5)

imgs, _, labels = next(iter(test_loader))

img, lbls = test_img(test_loader, fcn_model_bs)
print(lbls.size())
imshow_test(lbls.squeeze(0).long().cpu(), original_image, 'FCN_Base.png')
rprt = evaluate(val_ldr_pln, fcn_model_bs)
for ky in rprt.keys():
  print(ky, np.nanmean(rprt[ky]))